## orderBy / sort

- orderBy와 sort는 똑같은 함수 원하는 것을 써도 된다
- 오름수찬, 내림차순 쓸때는 꼭 col객체 필요!
- asc : 오름차순
- desc : 내림차순

- 스파크에서 정렬하면 기본적으로 Null이 작은값으로 인식, -> 오름차순 맨앞, 내림차순 맨뒤
- 위치가 마음에 안든다면 강제로 지정
- asc_nulls_last, desc_nulls_firs

In [1]:
from pyspark.sql import (
    Row,
    SparkSession)
import pyspark.sql.functions as F

In [2]:
spark=(
    SparkSession
    .builder
    .appName("groupby_study")
    .master("spark://spark-master:7077")
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/29 09:25:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df=spark.read.csv(
    "file:///workspace/data/movies_null.csv",
    header=True,
    inferSchema=True
)
df.show()
df.printSchema()

+--------+------------+---------+------------+------+-------+
|movie_id|       title|    genre|release_year|rating|revenue|
+--------+------------+---------+------------+------+-------+
|       1|   Inception|       SF|        2010|   8.8|    829|
|       2|    Parasite|    Drama|        2019|   8.6|    258|
|       3|    Avengers|   Action|        2012|  NULL|   1518|
|       4|Interstellar|       SF|        2014|   8.6|   NULL|
|       5|       Joker|    Drama|        2019|   8.4|   1074|
|       6|    ToyStory|Animation|        1995|  NULL|    394|
|       7|       Tenet|       SF|        2020|   7.4|   NULL|
|       8|   Gladiator|   Action|        2000|   8.5|    460|
+--------+------------+---------+------------+------+-------+

root
 |-- movie_id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- release_year: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- revenue: integer (nullable = true)



In [4]:
# 내림차순 오름차순시 반드시 col객체 필요!

In [5]:
df.orderBy("rating").show()

+--------+------------+---------+------------+------+-------+
|movie_id|       title|    genre|release_year|rating|revenue|
+--------+------------+---------+------------+------+-------+
|       3|    Avengers|   Action|        2012|  NULL|   1518|
|       6|    ToyStory|Animation|        1995|  NULL|    394|
|       7|       Tenet|       SF|        2020|   7.4|   NULL|
|       5|       Joker|    Drama|        2019|   8.4|   1074|
|       8|   Gladiator|   Action|        2000|   8.5|    460|
|       2|    Parasite|    Drama|        2019|   8.6|    258|
|       4|Interstellar|       SF|        2014|   8.6|   NULL|
|       1|   Inception|       SF|        2010|   8.8|    829|
+--------+------------+---------+------------+------+-------+



In [6]:
# df.orderBy("rating").desc().show()

In [7]:
df.orderBy(F.col("rating").desc()).show()

+--------+------------+---------+------------+------+-------+
|movie_id|       title|    genre|release_year|rating|revenue|
+--------+------------+---------+------------+------+-------+
|       1|   Inception|       SF|        2010|   8.8|    829|
|       2|    Parasite|    Drama|        2019|   8.6|    258|
|       4|Interstellar|       SF|        2014|   8.6|   NULL|
|       8|   Gladiator|   Action|        2000|   8.5|    460|
|       5|       Joker|    Drama|        2019|   8.4|   1074|
|       7|       Tenet|       SF|        2020|   7.4|   NULL|
|       3|    Avengers|   Action|        2012|  NULL|   1518|
|       6|    ToyStory|Animation|        1995|  NULL|    394|
+--------+------------+---------+------------+------+-------+



In [8]:
# orderBy

In [9]:
df.select(
    F.col("title"),
    F.col("rating")
).orderBy(F.col("rating").asc()).show()

+------------+------+
|       title|rating|
+------------+------+
|    Avengers|  NULL|
|    ToyStory|  NULL|
|       Tenet|   7.4|
|       Joker|   8.4|
|   Gladiator|   8.5|
|    Parasite|   8.6|
|Interstellar|   8.6|
|   Inception|   8.8|
+------------+------+



In [10]:
#sort 

In [11]:
df.select(
    F.col("title"),
    F.col("rating")
).sort(F.col("rating").asc()).show()

+------------+------+
|       title|rating|
+------------+------+
|    Avengers|  NULL|
|    ToyStory|  NULL|
|       Tenet|   7.4|
|       Joker|   8.4|
|   Gladiator|   8.5|
|    Parasite|   8.6|
|Interstellar|   8.6|
|   Inception|   8.8|
+------------+------+



In [12]:
# 내림차순 desc

In [13]:
df.select(
    F.col("title"),
    F.col("rating")
).orderBy(F.col("rating").desc()).show()

+------------+------+
|       title|rating|
+------------+------+
|   Inception|   8.8|
|    Parasite|   8.6|
|Interstellar|   8.6|
|   Gladiator|   8.5|
|       Joker|   8.4|
|       Tenet|   7.4|
|    Avengers|  NULL|
|    ToyStory|  NULL|
+------------+------+



In [14]:
# 다중 정렬 
# 1차 기준으로 정렬하고 그 안에서 2차 기준으로 다시 정렬!

In [15]:
df.select(
    F.col("genre"),
    F.col("title"),
    F.col("rating")
).orderBy(
    F.col("genre").asc(), # <- 1차기준
    F.col("rating").desc() # <- 2차기준
).show()

+---------+------------+------+
|    genre|       title|rating|
+---------+------------+------+
|   Action|   Gladiator|   8.5|
|   Action|    Avengers|  NULL|
|Animation|    ToyStory|  NULL|
|    Drama|    Parasite|   8.6|
|    Drama|       Joker|   8.4|
|       SF|   Inception|   8.8|
|       SF|Interstellar|   8.6|
|       SF|       Tenet|   7.4|
+---------+------------+------+



In [16]:
# null

In [21]:
df.select(
    F.col("title"),
    F.col("rating")
).orderBy(F.col("rating").asc_nulls_last()).show()

+------------+------+
|       title|rating|
+------------+------+
|       Tenet|   7.4|
|       Joker|   8.4|
|   Gladiator|   8.5|
|    Parasite|   8.6|
|Interstellar|   8.6|
|   Inception|   8.8|
|    Avengers|  NULL|
|    ToyStory|  NULL|
+------------+------+



In [24]:
df.select(
    F.col("title"),
    F.col("rating")
).orderBy(F.col("rating").desc_nulls_first()).show()

+------------+------+
|       title|rating|
+------------+------+
|    Avengers|  NULL|
|    ToyStory|  NULL|
|   Inception|   8.8|
|    Parasite|   8.6|
|Interstellar|   8.6|
|   Gladiator|   8.5|
|       Joker|   8.4|
|       Tenet|   7.4|
+------------+------+

